# Wildland Fire Data Analysis on Healthcare Impact

The goal of this notebook is to examine the health impact of wildfires on Indianapolis, IN. More specically, we will be assessing the impact of wildre smoke on respiratory health.

## License

The code lies under the standard [MIT licence](./LICENSE).

## Setup

In [1]:
#########
#
#    IMPORTS
#

# In-built libraries
import os
import json

# Third-party libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

In [ ]:
#########
#
#    CONSTANTS
#
FIRE_ANNUAL_SMOKE_ESTIMATES_PATH = "final/fire_annual_smoke_estimates.csv"
AQI_ANNUAL_ESTIMATES_PATH = "final/aqi_annual_estimates.csv"
HEALTHCARE_COMBINED_DATA_PATH = 'intermediate/healthcare_combined_data.csv'

## Exploratory Data Analysis

In [3]:
fire_annual_smoke_estimates_df = pd.read_csv(FIRE_ANNUAL_SMOKE_ESTIMATES_PATH)
aqi_annual_smoke_estimates_df = pd.read_csv(AQI_ANNUAL_ESTIMATES_PATH)

,Fire_Year,smoke_estimate_weighted
0,1964,0.385171
1,1965,0.431442
